https://adventofcode.com/2020/day/11

In [ ]:
file = '2020-12-11_test.txt'  #2020-12-11_input.txt
with open(file) as f:
    records = f.read().split('\n')
    
records = [x for x in records[:-1]]
len(records)

## First part

In [ ]:
import numpy as np
def split(line): 
    return [char for char in line]  
a = np.array([split(x) for x in records])
a = np.where(a=='.', np.nan, a)
a = np.where(a=='L', 0, a)
a = a.astype(np.float32)
a
b = np.zeros([a.shape[0],a.shape[1],2])
b[:,:,0] = a

In [ ]:
def assess(x,y,z):
    x1 = max(x-1,0)
    x2 = min(x+2,a.shape[0])
    y1 = max(y-1,0)
    y2 = min(y+2,a.shape[1])
    window = b[x1:x2,y1:y2,z]
    #print(window)
    
    if np.isnan(b[x,y,z]):
        result = np.nan
    elif b[x,y,z] == 0:
        result = int(np.nansum(window) < 1) 
    elif b[x,y,z] == 1:
        result = int(np.nansum(window) < 5)
        
    return result


In [ ]:
it_changes = True
c = 0
seated = 0
while it_changes:
    zs = c%2 # source
    c+=1
    zt = abs(zs-1)
    for x in range(b.shape[0]):
        for y in range(b.shape[1]):
            b[x,y,zt] = assess(x,y,zs)
    prev_seated = seated
    seated = np.nansum(b[:,:,zt])
    print(seated)
    if prev_seated == seated:
        print('stabilized')
        it_changes = False

## Second part

In [1]:
file = '2020-12-11_test.txt'  #2020-12-11_input.txt
with open(file) as f:
    records = f.read().split('\n')
    records = records[:-1] # cut empty line
len_x = len(records)
len_y = len(records[0])
flat_records = ''.join(records)
print(len_x,len_y)

10 10


In [2]:
def north(p,x,y):
    while (p-x) > x: 
        # as long as we are not in the first row
        p = p-x
        if val[p] >= 0:
#            return f'n{p}'
            return p

def north_east(p,x,y):
    while ((p-x)>x and (p+1)%x > 0):
        p = p-x+1
        if val[p] >= 0:
#            return f'ne{p}'
            return p

def east(p,x,y): 
    while ((p+1)%x > 0): 
        # as long as we are not in the first column
        p = p+1
        if val[p] >= 0:
#            return f'e{p}'
            return p

def south_east(p,x,y):
    while ((p+1)%x > 0 and (p+x) <= x*(y-1)):
        p = p+x+1
        if val[p] >= 0:
#            return f'se{p}'
            return p
    
def south(p,x,y):
    while (p+x) <= x*(y-1): 
        # as long as we are not in the last row
        p = p+x
        if val[p] >= 0:
#            return f's{p}'
            return p

def south_west(p,x,y):
    while ((p+x) <= x*(y-1) and ((p%x) > 0)):
        p = p+x-1
        if val[p] >= 0:
#            return f'sw{p}'
            return p

def west(p,x,y):
    while ((p%x) > 0): 
        # as long as we are not in the last column
        p = p-1
        if val[p] >= 0:
#            return f'w{p}'
            return p

def north_west(p,x,y):
    while ((p-x) > x and ((p%x) > 0)):
        p = p-x-1
        if val[p] >= 0:
#            return f'nw{p}'
            return p


In [3]:
val = []
for i in flat_records:
    if i == 'L':
        val.append(0)
    if i == '.':
        val.append(-1)
ref = []
x = len_x
y = len_y
for i in range(len(val)):
    tmp = []
    tmp.append(north(i,x,y))
    tmp.append(north_east(i,x,y))
    tmp.append(east(i,x,y))
    tmp.append(south_east(i,x,y))
    tmp.append(south(i,x,y))
    tmp.append(south_west(i,x,y))
    tmp.append(west(i,x,y))
    tmp.append(north_west(i,x,y))
    ref.append([x for x in tmp if x is not None])

In [ ]:
[i for i in range(len(val)) if val[i] >= 0]

In [ ]:
sum([i for i in val if i >= 0])

In [16]:
tmp = val.copy()
for j in range(len(val)):
    if (val[j] == 1):
        if (sum([val[i] for i in ref[j]]) >= 5):
            tmp[j] = 0
    elif (val[j] == 0):
        if (sum([val[i] for i in ref[j]]) == 0):
            tmp[j] = 1
val = tmp.copy()


In [17]:
sum([i for i in val if i >= 0])

30